In [1]:
#import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import dask.dataframe as dd
from IPython.display import display, HTML
import pandas as pd
import cudf

In [2]:
# df= cudf.read_csv('evaluation.csv')
# columns_to_remove = [
#     'customer_merchant_02', 'customer_merchant_01',
#     'customer_digital_activity_07', 'customer_digital_activity_08',
#     'customer_digital_activity_09', 'customer_digital_activity_18', 'customer', 'merchant'
# ]

# # Assuming 'data' is your cuDF DataFrame
# df.drop(columns=columns_to_remove, inplace=True)

# initial_parquet_path = 'imputed_test_df_0.parquet'
# df.to_parquet(initial_parquet_path)


In [3]:
import os
import joblib

def batch_impute_and_save_testset(test_df_path, model_directory, batch_size, base_filename, progress_tracker_file):
    """
    Perform or continue imputation on batches of columns for the test dataset using pre-trained models and save intermediate results.
    
    Args:
    test_df_path (str): Path to the test dataset (Parquet file) to impute.
    model_directory (str): Directory where the pre-trained models are saved.
    numerical_categories (list): List of columns to impute.
    exclude_columns (list): List of columns to exclude from imputation.
    batch_size (int): Number of columns to impute per batch.
    base_filename (str): Base name for saving intermediate Parquet files.
    progress_tracker_file (str): File to track the progress of imputation.
    """
    # Determine start index based on progress tracker file
    start_index = 0
    if os.path.exists(progress_tracker_file):
        with open(progress_tracker_file, 'r') as f:
            start_index = int(f.read().strip())
        test_df_path = f"test/{base_filename}_{start_index}.parquet"
    
    # Load the DataFrame
    df = cudf.read_parquet(test_df_path)

    exclude_columns = ['distance_05']
    categories = df.columns.to_list()
    for i in exclude_columns:
        categories.remove(i)
    
    for i, col in enumerate(categories[start_index:], start=start_index):
        if col in exclude_columns:
            continue  # Skip excluded columns
        
        model_path = f"{model_directory}/{col}_model.pkl"
        if not os.path.exists(model_path):
            print(f"No model found for column {col}, skipping.")
            continue

        # Load the model just-in-time
        model = joblib.load(model_path)
        print(f"Start imputing for column: {col}")
        # Impute missing values for the column
        missing_mask = df[col].isnull()
        if missing_mask.sum() > 0:
            X_missing = df.loc[missing_mask].drop(columns=[col]).fillna(0)
            df.loc[missing_mask, col] = model.predict(X_missing)
        
        print(f'Successfully imputed values for column: {col}')
        
        # Save after every batch or at the end
        if (i + 1) % batch_size == 0 or i == len(categories) - 1:
            save_path = f'test/{base_filename}_{i + 1}.parquet'
            df.to_parquet(save_path)
            with open(progress_tracker_file, 'w') as f:
                f.write(str(i + 1))
            print(f'Saved intermediate results to {save_path}')
            if i < len(categories) - 1:
                return  # Allows the process to be paused and resumed

    print("Imputation completed.")

# Parameters for the function call
test_df_path = 'test/imputed_test_df_0.parquet'
model_directory = 'impute_models'
batch_size = 15
base_filename = 'imputed_test_df'
progress_tracker_file = 'test_imputation_progress.txt'

# Start or continue the imputation
batch_impute_and_save_testset(test_df_path, model_directory, batch_size, base_filename, progress_tracker_file)

Imputation completed.
